# Visualizing Sentence Embeddings

In [10]:
import os
import sys
import numpy as np
import tensorflow as tf

from tensorflow.contrib.tensorboard.plugins import projector

# sys.path.append("~/neuralmonkey")

from neuralmonkey.experiment import Experiment
from neuralmonkey.run import load_runtime_config
from neuralmonkey import dataset

## 1. Load Neural Monkey model

In [11]:
exp = Experiment("mmmt-en-de/experiment.ini")
exp.build_model()

2018-08-25 17:25:11: Loading INI file: 'mmmt-en-de/experiment.ini'
2018-08-25 17:25:11: INI file is parsed.
2018-08-25 17:25:11: Building model based on the config.
2018-08-25 17:25:11: Vocabulary from wordlist loaded, containing 12142 words
2018-08-25 17:25:11: Sample of the vocabulary: ['hard-hat', 'pineapple', 'devil', 'ons', 'boxes']
2018-08-25 17:25:11: Vocabulary from wordlist loaded, containing 30003 words
2018-08-25 17:25:11: Sample of the vocabulary: ['abgesägten', 'Kniestrümpfen', 'X', 'Münztelefon', 'Hunderennens']
2018-08-25 17:25:11: Initializing decoder, name: 'decoder'
2018-08-25 17:25:11: Using linear projection of encoders as the initial state
2018-08-25 17:25:11: No output projection specified - using tanh projection
2018-08-25 17:25:12: The inferred rnn_size of this encoder projection will be 600
2018-08-25 17:25:12: Decoder initalized. Cost var: Tensor("decoder/Mean:0", shape=(), dtype=float32)
2018-08-25 17:25:12: Runtime logits tensor: Tensor("decoder/while_1/Exit

## 2. Load data

In [12]:
run_cfg = load_runtime_config("val-data.ini")
data = run_cfg.test_datasets[0]

2018-08-25 17:25:12: Loading INI file: 'val-data.ini'
2018-08-25 17:25:12: INI file is parsed.
2018-08-25 17:25:12: Building model based on the config.
2018-08-25 17:25:12: Initializing dataset with: target, source
2018-08-25 17:25:12: Dataset length: 1014
2018-08-25 17:25:12: Model built.


## 3. Run model on data

In [13]:
_, outputs = exp.run_model(data, batch_size=256)

INFO:tensorflow:Restoring parameters from mmmt-en-de/variables.data


2018-08-25 17:25:12: Default variable file 'mmmt-en-de/variables.data' will be used for loading variables.
2018-08-25 17:25:12: Loading variables from mmmt-en-de/variables.data
2018-08-25 17:25:14: Variables loaded from mmmt-en-de/variables.data


In [14]:
outputs['encoded'].shape

(1014, 600)

## 4. Visualize data in Projector

In [15]:
# Save embeddings in TSV format
np.savetxt("mmmt-en-de/embeddings.tsv", outputs['encoded'], delimiter='\t')

In [16]:
# Save source and output sentences in TSV format
with open("mmmt-en-de/metadata.tsv", "w") as f:
    # TSV header
    print("source\toutput", file=f)

    # Data
    for src_words, tgt_words in zip(data.get_series('source'), outputs['target']):
        src = ' '.join(src_words)
        tgt = ' '.join(tgt_words)
        print("{}\t{}".format(src, tgt), file=f)

In [17]:
# Create Projector config object
config = projector.ProjectorConfig()
emb = config.embeddings.add()
emb.tensor_name = "sent_embeddings"
emb.tensor_path = "embeddings.tsv"
emb.metadata_path = "metadata.tsv"

# Write Projector config to TensorBoard
writer = tf.summary.FileWriter("mmmt-en-de")
projector.visualize_embeddings(writer, config)